# 1) Preparación previa

### Carga de librerías

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import scipy as sp
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

### Lectura del data set limpio

In [ ]:
data = pd.read_csv("data_final.csv", sep = ";")
data.head(5)

# 2) Creación de dummies

### Partido

In [ ]:
# Como hay muchísimos, se usará el dataset con solo los 50 partidos con más datos
grupo_partidos = data.groupby("Partido")
filtro_partidos = grupo_partidos[["Partido"]].describe()
data_partidos = filtro_partidos[filtro_partidos.Partido.freq > 289]
len(data_partidos)

In [ ]:
mask_partidos = data['Partido'].isin(data_partidos.Partido.top)
data_para_dummies = data[mask_partidos]
data_para_dummies

In [ ]:
dummy_partido = pd.get_dummies(data_para_dummies['Partido'], prefix='partido')
dummy_partido

### Propiedades

In [ ]:
dummy_prop = pd.get_dummies(data_para_dummies['property_type'], prefix='prop')
dummy_prop

### Ambientes

In [ ]:
dummy_amb = pd.get_dummies(data_para_dummies['ambientes_final'], prefix='amb')
dummy_amb

### Balcón

In [ ]:
dummy_balcon = pd.get_dummies(data_para_dummies['balcon'], prefix = "dummy")
dummy_balcon

### Parrilla

In [ ]:
dummy_parrilla = pd.get_dummies(data_para_dummies['parrilla'], prefix = "dummy")
dummy_parrilla

### Pileta

In [ ]:
dummy_pileta = pd.get_dummies(data_para_dummies['pileta'], prefix = "dummy")
dummy_pileta

### Patio

In [ ]:
dummy_patio = pd.get_dummies(data_para_dummies['patio'], prefix = "dummy")
dummy_patio

### Quincho

In [ ]:
dummy_quincho = pd.get_dummies(data_para_dummies['quincho'], prefix = "dummy")
dummy_quincho

### Gimnasio

In [ ]:
dummy_gimnasio = pd.get_dummies(data_para_dummies['gimnasio'], prefix = "dummy")
dummy_gimnasio

### SUM

In [ ]:
dummy_sum = pd.get_dummies(data_para_dummies['sala_usos_multiples'], prefix = "dummy")
dummy_sum

### Cochera

In [ ]:
dummy_cochera = pd.get_dummies(data_para_dummies['cochera'], prefix = "dummy")
dummy_cochera

### Seguridad

In [ ]:
dummy_seguridad = pd.get_dummies(data_para_dummies['seguridad'], prefix = "dummy")
dummy_seguridad

### Jardín

In [ ]:
dummy_jardin = pd.get_dummies(data_para_dummies['jardin'], prefix = "dummy")
dummy_jardin

### Frente

In [ ]:
dummy_frente = pd.get_dummies(data_para_dummies['frente'], prefix = "dummy")
dummy_frente

In [ ]:
dummy_amenities = dummy_balcon.dummy_balcon + dummy_parrilla.dummy_parrilla + dummy_pileta.dummy_pileta + dummy_patio.dummy_patio + dummy_quincho.dummy_quincho + dummy_gimnasio.dummy_gimnasio + dummy_sum.dummy_sum + dummy_cochera.dummy_cochera + dummy_seguridad.dummy_seguridad + dummy_jardin.dummy_jardin + dummy_frente.dummy_frente
dummy_amenities.name = "dummy_amenities"

In [ ]:
dummy_amenities.value_counts()

### Unificación de dummies en un dataset

In [ ]:
data_con_dummies = pd.concat([data_para_dummies, dummy_amenities, dummy_partido, dummy_prop, dummy_amb],axis=1)
data_con_dummies.columns

# 3) Creación de modelo uninominal

## Aplicación en todo el dataset

In [ ]:
X_train_todo_dataset = data_con_dummies[data_con_dummies.ambientes_imputados == 0].drop(["property_type", "price", "surface_covered_in_m2", "Partido", "ambientes_train",
                                                                          "ambientes_imputados", "ambientes_final", "balcon", "parrilla", "pileta", "patio",
                                                                          "quincho", "gimnasio", "sala_usos_multiples", "cochera", "seguridad", "jardin", "frente"], axis = 1)
X_test_todo_dataset = data_con_dummies[data_con_dummies.ambientes_train == 0].drop(["property_type", "price", "surface_covered_in_m2", "Partido", "ambientes_train",
                                                                          "ambientes_imputados", "ambientes_final", "balcon", "parrilla", "pileta", "patio",
                                                                          "quincho", "gimnasio", "sala_usos_multiples", "cochera", "seguridad", "jardin", "frente"], axis = 1)
Y_train_todo_dataset = X_train_todo_dataset["precio_usd_por_m2"]
Y_test_todo_dataset = X_test_todo_dataset["precio_usd_por_m2"]

X_train_todo_dataset.drop(["precio_usd_por_m2"], axis = 1, inplace = True)
X_test_todo_dataset.drop(["precio_usd_por_m2"], axis = 1, inplace = True)

In [ ]:
data_con_dummies.shape

In [ ]:
X_train_todo_dataset.columns

#### Creación de los elementos a utilizar

In [ ]:
lm_todo_dataset = linear_model.LinearRegression()
# Fiteamos el modelo sobre los vectores X e Y.
model_todo_dataset = lm_todo_dataset.fit(X_train_todo_dataset, Y_train_todo_dataset)

#### Aplicación del modelo en los datos

In [ ]:
# Guardamos  las predicciones en un nuevo vector que llamaremos predictions.
predictions_todo_dataset = lm_todo_dataset.predict(X_test_todo_dataset)

# Imprimimos el intercepto y los coeficientes como atributos del objeto entrenado.
print ('Intercepto =', model_todo_dataset.intercept_)
print ('RM =', model_todo_dataset.coef_)

# Imprimos la metrica que mide la bondad de ajusto del modelo. En este caso el R2.
print ('R2_train =', model_todo_dataset.score(X_train_todo_dataset, Y_train_todo_dataset))

#### Obtención de métricas

In [ ]:
#media del precio_usd_por_m2
data_con_dummies.precio_usd_por_m2.mean()

In [ ]:
# Error absoluto de la media
mean_absolute_error(Y_test_todo_dataset, predictions_todo_dataset)

In [ ]:
# Raiz cuadrada de la media del error
mean_squared_error(Y_test_todo_dataset, predictions_todo_dataset)

In [ ]:
# Raiz cuadrada de la media del error
RMSE = np.sqrt(mean_squared_error(Y_test_todo_dataset, predictions_todo_dataset))
RMSE

#### Análisis para Regresion Lineal

En primer lugar, se obtiene el conjunto de datos residuales y se los grafica para observar su distribución.

In [ ]:
test_residuals = Y_test_todo_dataset - predictions_todo_dataset

In [ ]:
sns.distplot(test_residuals, kde = True)

La distribucion de los Errores Residuales pareciera ser una Normal con media en 0. Igualmente, podría haber ser un caso del Cuarteto de Anscombe por lo que se procede graficando la probabilidad en torno a los quantiles.

In [ ]:
fig,ax = plt.subplots(figsize = (3,4), dpi = 100)
sp.stats.probplot(test_residuals,plot = ax)

Con este gráfico se corrobora efectivamente que la distribución de los Errores Residuales se aproxima a una Distribución Normal y, por ende, se puede proceder con la creación de la regresion lineal.

In [ ]:
sns.scatterplot(x = Y_test_todo_dataset, y = test_residuals)
plt.axhline(y = 0, color = 'r', ls = "--")
plt.xlabel("Precio por M2 en dólares del Y test")
plt.ylabel("Error residual de Y test en precio por M2 en dólares")

## Regresión lineal entre superficie cubierta y precio por m2 en dólares

In [ ]:
X_train_superficie = data_con_dummies[data_con_dummies.ambientes_imputados == 0].drop(['property_type', 'price', 'Partido', 'ambientes_train', 'ambientes_imputados',
       'ambientes_final', 'balcon', 'parrilla', 'pileta', 'patio', 'quincho',
       'gimnasio', 'sala_usos_multiples', 'cochera', 'seguridad', 'jardin',
       'frente', 'dummy_amenities', 'partido_Almagro',
       'partido_Almirante Brown', 'partido_Avellaneda', 'partido_Bahía Blanca',
       'partido_Balvanera', 'partido_Barracas', 'partido_Barrio Norte',
       'partido_Belgrano', 'partido_Boedo', 'partido_Caballito',
       'partido_Colegiales', 'partido_Córdoba', 'partido_Escobar',
       'partido_Esteban Echeverría', 'partido_Ezeiza', 'partido_Flores',
       'partido_Floresta', 'partido_General San Martín', 'partido_Ituzaingó',
       'partido_La Matanza', 'partido_La Plata', 'partido_Lanús',
       'partido_Lomas de Zamora', 'partido_Mar del Plata', 'partido_Monserrat',
       'partido_Moreno', 'partido_Morón', 'partido_Nuñez', 'partido_Palermo',
       'partido_Pilar', 'partido_Pinamar', 'partido_Punilla',
       'partido_Quilmes', 'partido_Recoleta', 'partido_Rosario',
       'partido_Saavedra', 'partido_San Cristobal', 'partido_San Fernando',
       'partido_San Isidro', 'partido_San Miguel', 'partido_San Telmo',
       'partido_Tigre', 'partido_Tres de Febrero', 'partido_Vicente López',
       'partido_Villa Carlos Paz', 'partido_Villa Crespo',
       'partido_Villa Devoto', 'partido_Villa Luro', 'partido_Villa Urquiza',
       'partido_Villa del Parque', 'prop_PH', 'prop_apartment', 'prop_house',
       'amb_1.0', 'amb_2.0', 'amb_3.0', 'amb_4.0', 'amb_5.0', 'amb_6.0',
       'amb_7.0'], axis = 1)
X_test_superficie = data_con_dummies[data_con_dummies.ambientes_train == 0].drop(['property_type', 'price', 'Partido', 'ambientes_train', 'ambientes_imputados',
       'ambientes_final', 'balcon', 'parrilla', 'pileta', 'patio', 'quincho',
       'gimnasio', 'sala_usos_multiples', 'cochera', 'seguridad', 'jardin',
       'frente', 'dummy_amenities', 'partido_Almagro',
       'partido_Almirante Brown', 'partido_Avellaneda', 'partido_Bahía Blanca',
       'partido_Balvanera', 'partido_Barracas', 'partido_Barrio Norte',
       'partido_Belgrano', 'partido_Boedo', 'partido_Caballito',
       'partido_Colegiales', 'partido_Córdoba', 'partido_Escobar',
       'partido_Esteban Echeverría', 'partido_Ezeiza', 'partido_Flores',
       'partido_Floresta', 'partido_General San Martín', 'partido_Ituzaingó',
       'partido_La Matanza', 'partido_La Plata', 'partido_Lanús',
       'partido_Lomas de Zamora', 'partido_Mar del Plata', 'partido_Monserrat',
       'partido_Moreno', 'partido_Morón', 'partido_Nuñez', 'partido_Palermo',
       'partido_Pilar', 'partido_Pinamar', 'partido_Punilla',
       'partido_Quilmes', 'partido_Recoleta', 'partido_Rosario',
       'partido_Saavedra', 'partido_San Cristobal', 'partido_San Fernando',
       'partido_San Isidro', 'partido_San Miguel', 'partido_San Telmo',
       'partido_Tigre', 'partido_Tres de Febrero', 'partido_Vicente López',
       'partido_Villa Carlos Paz', 'partido_Villa Crespo',
       'partido_Villa Devoto', 'partido_Villa Luro', 'partido_Villa Urquiza',
       'partido_Villa del Parque', 'prop_PH', 'prop_apartment', 'prop_house',
       'amb_1.0', 'amb_2.0', 'amb_3.0', 'amb_4.0', 'amb_5.0', 'amb_6.0',
       'amb_7.0'], axis = 1)
Y_train_superficie = X_train_superficie["precio_usd_por_m2"]
Y_test_superficie = X_test_superficie["precio_usd_por_m2"]

X_train_superficie.drop(["precio_usd_por_m2"], axis = 1, inplace = True)
X_test_superficie.drop(["precio_usd_por_m2"], axis = 1, inplace = True)

In [ ]:
lm_superficie = linear_model.LinearRegression()
# Fiteamos el modelo sobre los vectores X e Y.
model_superficie = lm_superficie.fit(X_train_superficie, Y_train_superficie)

In [ ]:
# Guardamos  las predicciones en un nuevo vector que llamaremos predictions.
predictions_superficie = lm_superficie.predict(X_test_superficie)

# Imprimimos el intercepto y los coeficientes como atributos del objeto entrenado.
print ('Intercepto =', model_superficie.intercept_)
print ('RM =', model_superficie.coef_)

# Imprimos la metrica que mide la bondad de ajusto del modelo. En este caso el R2.
print ('R2_train =', model_superficie.score(X_train_superficie, Y_train_superficie))

In [ ]:
test_residuals_superficie = Y_test_superficie - predictions_superficie

In [ ]:
sns.scatterplot(x = Y_test_superficie, y = test_residuals_superficie)
plt.axhline(y = 0, color = 'r', ls = "--")
plt.xlabel("Precio por M2 en dólares del Y test")
plt.ylabel("Error residual de Y test en precio por M2 en dólares")

In [ ]:
# Imprimos la metrica que mide la bondad de ajusto del modelo. En este caso el R2.
R2_train_superficie = model_superficie.score(X_train_superficie, Y_train_superficie)
R2_train_superficie

In [ ]:
# Error absoluto de la media
MAE_Test_superficie = mean_absolute_error(Y_test_superficie, predictions_superficie)
MAE_Test_superficie

In [ ]:
# Raiz cuadrada de la media del error
MSE_Test_superficie = mean_squared_error(Y_test_superficie, predictions_superficie)
MSE_Test_superficie

In [ ]:
RMSE_Test_superficie = np.sqrt(mean_squared_error(Y_test_superficie, predictions_superficie))
RMSE_Test_superficie

Se crea un diccionario con los datos que se recopilan de ahora en adelante, así se comparan al final de la notebook 3

In [ ]:
revision_datos = [ {'subconjunto': 'Superficie Cubierta',
    'R2_train': R2_train_superficie.round(4),
    'MAE': MAE_Test_superficie.round(4),
    'MSE': MSE_Test_superficie.round(4),
    'RMSE': RMSE_Test_superficie.round(4)}]

# 5) Creación de modelo polinomial

In [ ]:
poly_train = PolynomialFeatures(2)
poly_features_train = poly_train.fit_transform(X_train_todo_dataset)

In [ ]:
poly_test = PolynomialFeatures(2)
poly_features_test = poly_test.fit_transform(X_test_todo_dataset)

In [ ]:
# X_train_todo_dataset, X_test_todo_dataset, Y_train_todo_dataset, Y_test_todo_dataset = train_test_split (poly_features, Y_train_todo_dataset, test_size = 0.3, random_state = 101)

In [ ]:
poly_model = linear_model.LinearRegression()

In [ ]:
poly_predictions = poly_model.fit(poly_features_train, Y_train_todo_dataset).predict(poly_features_test)

In [ ]:
print ('R2_train =', poly_model.score(poly_features_train, Y_train_todo_dataset))

In [ ]:
poly_model.coef_

In [ ]:
MAE = mean_absolute_error(Y_test_todo_dataset, poly_predictions)
MAE

In [ ]:
MSE = mean_squared_error(Y_test_todo_dataset, poly_predictions)
MSE

In [ ]:
RMSE = np.sqrt(MSE)
RMSE

Exportamos el dataset para hacer gráficos y comparaciones

In [ ]:
data_con_dummies.to_csv('data_con_dummies.csv', index = False, sep=';')